In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np
from tensorflow import keras



In [8]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'data',
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


Found 30000 files belonging to 25 classes.
Using 24000 files for training.
Found 30000 files belonging to 25 classes.
Using 6000 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)  # Check the class names

['Asian-Green-Bee-Eater', 'Brown-Headed-Barbet', 'Cattle-Egret', 'Common-Kingfisher', 'Common-Myna', 'Common-Rosefinch', 'Common-Tailorbird', 'Coppersmith-Barbet', 'Forest-Wagtail', 'Gray-Wagtail', 'Hoopoe', 'House-Crow', 'Indian-Grey-Hornbill', 'Indian-Peacock', 'Indian-Pitta', 'Indian-Roller', 'Jungle-Babbler', 'Northern-Lapwing', 'Red-Wattled-Lapwing', 'Ruddy-Shelduck', 'Rufous-Treepie', 'Sarus-Crane', 'White-Breasted-Kingfisher', 'White-Breasted-Waterhen', 'White-Wagtail']


In [12]:
from keras import layers

normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Optionally, apply augmentation:
'''data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))'''


"data_augmentation = tf.keras.Sequential([\n    layers.RandomFlip('horizontal'),\n    layers.RandomRotation(0.1),\n])\ntrain_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))"

In [14]:
from keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
print(train_ds)  # Inspect the dataset object


<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [16]:
model.fit(train_ds, validation_data=val_ds, epochs=10)
model.save('my_custom_model.h5')


Epoch 1/10


2024-12-06 16:02:41.127394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


750/750 ━━━━━━━━━━━━━━━━━━━━ 78s 97ms/step - accuracy: 0.0373 - loss: 3.2201 - val_accuracy: 0.0378 - val_loss: 3.2198
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 70s 93ms/step - accuracy: 0.0403 - loss: 3.2192 - val_accuracy: 0.0378 - val_loss: 3.2202
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 71s 94ms/step - accuracy: 0.0397 - loss: 3.2192 - val_accuracy: 0.0378 - val_loss: 3.2203
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 70s 93ms/step - accuracy: 0.0402 - loss: 3.2192 - val_accuracy: 0.0342 - val_loss: 3.2203
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 70s 93ms/step - accuracy: 0.0386 - loss: 3.2192 - val_accuracy: 0.0378 - val_loss: 3.2203
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 69s 92ms/step - accuracy: 0.0407 - loss: 3.2192 - val_accuracy: 0.0378 - val_loss: 3.2203
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 68s 91ms/step - accuracy: 0.0384 - loss: 3.2192 - val_accuracy: 0.0342 - val_loss: 3.2203
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 69s 92ms/step - accuracy: 0.0383 - loss: 3.2192 - val_accurac